# Detecting Fraud - An Exploratory Data Analysis

A methodical EDA (exploratory data analysis) is comprised of four steps.   

  - The first of these steps involves loading the data into memory and storing it in a dataframe.  When a dataset does not fit into memory there are alternative methods of doing this - though I am unsure of what those are.
  -  In step 12 we will make the data set a "tidy data" set by following the guidelines established by Hadley Wickham.
 http://vita.had.co.nz/papers/tidy-data.html
  - Step three will use descriptive statistics and exploratory visualizations to understand the data at a macro level.
  - In step four we will aggregate the data and explore the group properties in a much more detailed manner.

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
# Importing pandas 0.18.1
# Importing numpy 1.12.1

In [ ]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (12, 8)
matplotlib.pyplot.style.use('ggplot')
pd.set_option('max.rows', 100)

In [ ]:
pd.__version__

In [ ]:
np.__version__

In [ ]:
matplotlib.__version__

### Step 1:  Loading and storing data set into a data frame

In [ ]:
df = pd.read_csv('train.csv')
# importing preprocess to load 'train' dataset
# making dataframe df from 'train' dataset
# completes step 1 of a methodical EDA

### Step 2:  Making a tidy data set

In [ ]:
df.shape
# Curious as to the size of the data set

In [ ]:
df.head()
# curious as to the content of the data set

### Why look at the head of the data set?

I chose to begin step two of my EDA, tidying up the data set, by looking at the head of the data frame.  In doing this I can take a look at my column headers as well as the first 5 results where I am looking for the presence of the 3 indicators of a tidy data set:
- Each variable forms a column
- Each observation forms a row
- Each type of observational unit forms a table

Though not all of these are going to be appparent with only 5 observations listed, we can capture any of the obvious ones and correct them right away.

According to Hadley, the 5 most frequent causes of messy data are as follows:
- Column headers are values and not variable names
- Mulitple variables are stored in one column
- Variables are stored in both rows and colummns
- Observational units types of varying kinds are stored in the same table
- Multiple instances of a single observational type is found in numerous tables

At the on set alll of these stipulations seems to be in order.  I do however take issue with the ordering of the columns.  Fixed variables should come first (left-most) followed by the measured variables.  Currently, all we have are fixed variables and the flow of the columns makes sense to me with the exception of the 'id' column.  It is my belief that this is a unique identifier for each transaction and as such it makes more sense to me that this column be moved to the far left side of the data frame.  I've also corrected the column header 'oldbalanceOrg' to 'oldbalanceOrig' thinking that the typo ought to be corrected.

As the analysis continues to deepen I will continue to look for additional manners to tidy up my data set where it is warranted.

In [ ]:
df = df[['id', 'step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg',
         'newbalanceOrig', 'nameDest', 'oldbalanceDest', 'newbalanceDest',
         'isFraud']]
df.head()
# moving 'id' column to the left end of df

In [ ]:
df = df.rename(columns={'oldbalanceOrg': 'oldbalanceOrig'})
# correcting 'oldbalanceOrig' column header

In [ ]:
df.head()
# confirming my changes
# completes step 2 of a methodical EDA

### Step 3:  Using descriptive statistics and visualizations to begin forming an understanding of the data set - macro level.

### What can we learn from our tabular analysis?
By looking at the below tables we can learn from our data the following:
- There are 445,383 observations none of which have null values
- 576 transactions were fraudulant (276 cash_out & 300 transfer)
- There are 5 different transaction types for us to take into account.  The msot frequest of these transaction types is the Cash_Out type.
- Out of the 445,383 transactions we have 445,338 unique customers originating the transaction.  This means we have 45 repeat originators.  However, of these repeat originators none appear more than twice.
- The mean transaction amount was 178,918 with the largest transaction being 60,642,000.
- 50% of origin account new balances are 0, indicating they were drained, be it on purpose or by fraud
- 69% (174,616) of the origin accounts having new balances of 0 ened up that way through Cash_Out and Transfer transaction types
- 88% (150,954) of the destination accounts having new balances of 0 ended up that way through Payment transaction types
- These same accounts have old and new balances equal to each other indicating to me they may be acting like pass through accounts.
- The average transfer amount was 903,518.
- I prefer tables when analyzing data despite most peoples preference for graphs


In [ ]:
df.isnull().sum()
# no need for imputation

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode()

In [ ]:
from plotly import __version__
print(__version__)

In [ ]:
df.info()
# getting concise summary
# dtypes make sense

In [ ]:
df.describe(include='all')
# Getting summary statistics
# including non-integer d-types

In [ ]:
pd.crosstab(df.type, df.isFraud == 1)
# Looking at transaction tyoes for fraudulent transactions

In [ ]:
pd.crosstab(df.type, df.newbalanceOrig == 0.0)
# '''Looking at post transfer balances equal to $0
# of orgin accounts by transaction type'''

In [ ]:
pd.crosstab(df.type, df.newbalanceDest == 0.0)
# '''Looking at post transfer balances equal to $0
# of destination accounts by transaction type'''

In [ ]:
pd.crosstab(df.type, df.oldbalanceDest == df.newbalanceDest)
# '''Looking for destination accounts having pre
# and post transaction balances equal to each other'''

In [ ]:
pd.crosstab(df.type, df.amount == 0, values=df.amount, aggfunc=np.mean)
# Looking at average transaction amount by transaction type

### What can we learn from our analysis through visualization?
By looking at the below visualizations we can learn from our data the following:
- Due to the disparity in our data the ability to zoom offered through plotly is exeremely helpful here.  At the greatest zoom many of these charts are difficult to analyze.
- Graphics slowed my notebook
- Though large in transaction size the fraudulent transactions were not large in number

In [ ]:
trace = go.Histogram(x=df.oldbalanceOrig,
                     nbinsx=10, name='Old Balance',)

trace2 = go.Histogram(x=df.newbalanceOrig,
                      name='New Balance', opacity=0.95)
data = [trace, trace2]
layout = go.Layout(title='Origin Account Comparison',
                   xaxis=dict(title='Balance'),
                   yaxis=dict(title='Frequency',))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)
# Comparing balance of origin account
# pre- and post- transfer

In [ ]:
trace = go.Histogram(x=df.amount,
                     nbinsx=10, name='Transaction Amount',)
data = [trace]
layout = go.Layout(title='Transaction Amount',
                   xaxis=dict(title='Amount'),
                   yaxis=dict(title='Frequency',))
fig = go.Figure(data=data, layout=layout)
py.iplot(fig)
# Transaction Amount Frequency

In [ ]:
trace = go.Box(y=df.amount, name='Transaction Amount')
data = [trace]
py.iplot(data)
# box plot by transaction amount

In [ ]:
trace = go.Scatter(x=df.newbalanceDest,
                   y=df.amount, mode='markers',
                   marker=dict(color=df.isFraud, showscale=True))
data = [trace]
py.iplot(data)
# scatter plot of tranaction amount by type and colored by fraudulent or not

In [ ]:
annotations = [dict(x=xi, y=yi,
                    text=str(yi), xanchor='center',
                    yanchor='bottom', showarrow=False,)
               for yi, xi in zip(df.isFraud.value_counts(), ['0', '1'])]

In [ ]:
trace = go.Bar(
    y=df.isFraud.value_counts(),
    x=['0', '1'],)
layout = go.Layout(
    annotations=annotations)
data = [trace]
py.iplot(dict(data=data, layout=layout))
# bar chart transaction type count

In [ ]:
annotations = [dict(x=xi, y=yi, text=str(yi),
                    xanchor='center', yanchor='bottom',
                    showarrow=False,)
               for yi, xi in zip(df.type.value_counts(),
                                 ['cash_in', 'cash_out',
                                  'debit', 'transfer', 'payment'])]

In [ ]:
trace = go.Bar(y=df.type.value_counts(),
               x=['cash_in', 'cash_out', 'debit', 'transfer', 'payment'],)
layout = go.Layout(annotations=annotations)
data = [trace]
py.iplot(dict(data=data, layout=layout))
# bar chart transaction type count
# completes step 3 of a methodical EDA

### Step 4:  Aggregation of the data and exploration of the group properties at the micro level.

### What can we learn from our micro level analysis?
By looking at the below visualizations and tables we can learn from our data the following:
- There were 276 fraudulent transactions of the Cash_Out type with a mean amount of 1,587,794 in local currency.  Half of all fraudulent transactions of this type were in the amount of 407,495
- There were 300 fraudulent transactions of the Transfer type with a mean amount of 1,484,329 in local currency.  Half of all fraudulent transactions of this type were in the amount of 463,901
- Only 130 (47%) fruadulent transactions were below 100,000
- New balances for the destination accounts were 0 75% of the time for the fraudulent Transfer transaction types.  curious 

In [ ]:
df.groupby(['isFraud', 'type']).describe()
# looking at data by transaction type and fraudulent or not

In [ ]:
df.groupby(['type', df.amount < 100000]).isFraud.value_counts().unstack()
# determining fraudulent transactions over $100,000

In [ ]:
pd.crosstab(df.type, df.isFraud).plot.bar()
# completes step 4 of a methodical EDA

### What is my data coupled with my intuition telling me the story is here?
A typical fruadulent account is not frequent in nature occuring only about .1% of the all transactions.  When a fraudulent transaction ois successful however it is devistanting, draining it's victim's account balance to $0 in nealry every case.  The average fruadulent transaction amount being 1.5 million.  Of the 5 transaction types the two that are most susceptible are Cash Out and Transfer transactions.  The risk of a fraudulent Transfer transaction taking place actually increases to 1 percent.  Interestingly the ending balance on the destination accounts on Cash Out transactions seemingly represents the increase for the fruadulent funds deposited where in the case of the Transfer transactions the funds seem to disappear.  
